# Description

Writing parameterized decorators is a powerful way of describing some general cross-cutting behavior that is also partially specialized.  However, in decoratoring a function you have to determine if the "decorator" is actually a decorator or a decorator function.  At least you do in the examples in the lessons.

A number of widely used decorators in the standard library and in widely-used third-party tools can act as *either* a decorator factory *or* as a decorator directly.  For example:

```python
from functools import lru_cache

@lru_cache
def add1(a, b):
    return a+b

@lru_cache(maxsize=128)
def add2(a, b):
    return a+b
```

Both `add1()` and `add2()` are memoized, and in fact have the same maximum cache size.  In the factory variation, you could specify a non-default `maxsize`, of course.

For this exercise, you should write your own dual-use decorator.  Your will be called `@orbits` after the style of a Mandelbrot set function which resembles this:

```python
def mandelbrot(z0:complex, orbits:int=255) -> complex:
    z = z0
    for n in range(orbits):
        z = z**2 + z0
    return z
```

The idea here is that you would like to write a function that only expresses a single transformation of a (complex) number, but when decorated it will repeatedly apply that operation for a number of "orbits."  If not parameterized, it will default to 10 orbits.  As an example, the `m_orbit()` function is defined in the setup.

# Setup

In [1]:
# You may decide to use a class-based version.  
# Non-working identity decorator only created as example
def orbits(fn):
    def inner(z):
        return fn(z)
    return inner

# Definition of a single orbit
@orbits
def m_orbit(z: complex) -> complex:
    return z**2 + z

# correct is approx -0.0733106-0.0017179j
m_orbit(-0.6+0.3j)    #-> wrong is approx -0.33-0.06j

(-0.32999999999999996-0.06j)

# Solution

In [2]:
from types import FunctionType

def orbits(fun_or_count):
    if isinstance(fun_or_count, FunctionType):
        fn = fun_or_count
        def inner(z):
            for _ in range(10):
                z = fn(z)
            return z
        return inner
    
    elif isinstance(fun_or_count, int) and fun_or_count >= 0:
        n = fun_or_count
        def decorator(fn):
            def inner(z):
                for _ in range(n):
                    z = fn(z)
                return z
            return inner
        return decorator
    
    else:
        raise ValueError("Number of orbits must be a non-negative integer")

# Test Cases

In [3]:
def test_noparam():
    from cmath import isclose
    c = -0.1+0.65j
    @orbits
    def j_orbit(z):
        return z**2 + c
    z = j_orbit(0.1-0.5j)
    assert isclose(z, -0.3170447+0.5734000j, abs_tol=1e-7), z
    
test_noparam()

In [4]:
def test_param0():
    from cmath import isclose
    c = -0.1+0.65j
    @orbits(0)
    def j_orbit(z):
        return z**2 + c
    z = j_orbit(0.1-0.5j)    
    assert isclose(z, 0.1-0.5j), z
    
test_param0()

In [5]:
def test_param10():
    from cmath import isclose
    c = -0.1+0.65j
    @orbits(10)
    def j_orbit(z):
        return z**2 + c
    z = j_orbit(0.1-0.5j)
    assert isclose(z, -0.3170447+0.5734000j, abs_tol=1e-7), z
    
test_param10()

In [6]:
def test_param255():
    from cmath import isclose
    c = -0.1+0.65j
    @orbits(255)
    def j_orbit(z):
        return z**2 + c
    z = j_orbit(0.1-0.5j)
    assert isclose(z, -0.0304571+0.1706268j, abs_tol=1e-7), z
    
test_param255()

In [7]:
def test_default():
    from cmath import isclose
    c = -0.1+0.65j
    @orbits(10)
    def j1_orbit(z): return z**2 + c
    @orbits
    def j2_orbit(z): return z**2 + c
    assert j1_orbit(0.1-0.5j) == j2_orbit(0.1-0.5j)
    
test_default()